In [1]:
##will soon become a python file

import numpy as np
import gensim 
import nltk
import scipy
from scipy import spatial
import logging
from gensim.corpora.dictionary import Dictionary
from numpy import dot, zeros, dtype, float32 as REAL,\
    double, array, vstack, fromstring, sqrt, newaxis,\
    ndarray, sum as np_sum, prod, ascontiguousarray,\
    argmax
from pyemd import emd

C:\Users\Linda\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from nltk import word_tokenize
from nltk.corpus import stopwords

In [3]:
def stop_words(sentence,language):
    stopWords = set(stopwords.words('english'))
    wordsFiltered = []
    words = sentence.split(' ')
    for w in words:
        #remove manually words with single quote
        if w == "you're" or w == "i'll" or w == "we're" or w == "i'm" or w == "he's" or w == "she's" or w == "they're" :
            pass
        elif w not in stopWords:
            wordsFiltered.append(w)
    return wordsFiltered

In [4]:
#Load text and treat
def load_care(path): 
    text_en = []
    aux = open(path,encoding = 'utf8').readlines()
    for item in aux:
        item = item.lower()
        stringlist = []
        item = item.replace("'",'')
        for letter in item:  
            if letter == ',' or letter == '.' or letter == ';' or letter == '(' or letter == ')' or letter == '/' or letter == '"""' or letter == '"\"' or letter == '"' or letter == '?' or letter == '!':
                letter = ''
            elif letter.isdigit() == True:
                letter = ''
            stringlist.append(letter)
        text_en.append(''.join(stringlist))
        
    return text_en


In [5]:
src_path = '~/desktop/embeddings/wiki.en.align.vec'
tgt_path = '~/desktop/embeddings/wiki.de.align.vec'
nmax = 150000  
src_model = gensim.models.KeyedVectors.load_word2vec_format(src_path,limit = nmax)
tgt_model = gensim.models.KeyedVectors.load_word2vec_format(tgt_path,limit = nmax)

In [6]:
def embedder(sentence,model,language):                                                                     
    first_word = 0 
    word_emb = []
    sentence_test = stop_words(sentence,language)
    if len(sentence_test)>1:                              #do not apply stopwords for really short sentences
        sentence = sentence_test
    
    for word in sentence[:len(sentence)-2]:               #delete \n -> last 2 chars                                
        if first_word == 0: 
            if word in model:
                word_emb = model[word]
                first_word = 1
        else:
            if word in model:
                word_emb = word_emb + model[word]   
    
    return  word_emb    #np.append(word_emb,len(sentence))  returns the word embedding plus its size(300 + 1)
                        

In [7]:
def sentence_embedder(sentence_list,model,language):
    emb = []
    for sentence in sentence_list:
        save_last = sentence
        if(len(sentence)>1):
            emb.append(embedder(sentence,model,language))
    return emb

In [8]:
src_list = load_care('./sentences/src-train.txt')
tgt_list = load_care('./sentences/tgt-train.txt')

In [9]:
src_sentences = sentence_embedder(src_list[:100],src_model,'english')
tgt_sentences = sentence_embedder(tgt_list[:100],tgt_model,'german')

In [10]:
def generate_matrix(src_sentences,tgt_sentences):
    matriz = np.zeros((len(src_sentences),len(tgt_sentences)))
    for i in range(0,len(src_sentences)):
        listou = []
        for j in range(0,len(tgt_sentences)):
             listou.append(1-spatial.distance.cosine(src_sentences[i],tgt_sentences[j]))
        matriz = np.insert(matriz,i,listou,0)
        np.savetxt("result_matrix.csv",matriz, delimiter=",")




In [11]:
generate_matrix(src_sentences,tgt_sentences)

In [12]:
def wmdistance_b(src_model,tgt_model, document1, document2):
        """
        .. Ofir Pele and Michael Werman, "A linear time histogram metric for improved SIFT matching".
        .. Ofir Pele and Michael Werman, "Fast and robust earth mover's distances".
        .. Matt Kusner et al. "From Word Embeddings To Document Distances".
        Note that if one of the documents have no words that exist in the
        Word2Vec vocab, `float('inf')` (i.e. infinity) will be returned.
        This method only works if `pyemd` is installed (can be installed via pip, but requires a C compiler).
       """
        logger = logging.getLogger(__name__)
        PYEMD_EXT = True
        if not PYEMD_EXT:
            raise ImportError("Please install pyemd Python package to compute WMD.")

        # Remove out-of-vocabulary words.
        len_pre_oov1 = len(document1)
        len_pre_oov2 = len(document2)
        document1 = [token for token in document1 if token in src_model]
        document2 = [token for token in document2 if token in tgt_model]
        diff1 = len_pre_oov1 - len(document1)
        diff2 = len_pre_oov2 - len(document2)
        if diff1 > 0 or diff2 > 0:
            logger.info('Removed %d and %d OOV words from document 1 and 2 (respectively).', diff1, diff2)

        if len(document1) == 0 or len(document2) == 0:
            logger.info(
                "At least one of the documents had no words that werein the vocabulary. "
                "Aborting (returning inf)."
            )
            return float('inf')

        dictionary = Dictionary(documents=[document1, document2])
        vocab_len = len(dictionary)

        if vocab_len == 1:
            # Both documents are composed by a single unique token
            return 0.0

        # Sets for faster look-up.
        docset1 = set(document1)
        docset2 = set(document2)
        # Compute distance matrix.
        distance_matrix = zeros((vocab_len, vocab_len), dtype=double)
        for i, t1 in dictionary.items():
            for j, t2 in dictionary.items():
                if t1 not in docset1 or t2 not in docset2:
                    continue
                # Compute Euclidean distance between word vectors.
                distance_matrix[i, j] = sqrt(np_sum((src_model[t1] - tgt_model[t2])**2))

        if np_sum(distance_matrix) == 0.0:
            # `emd` gets stuck if the distance matrix contains only zeros.
            logger.info('The distance matrix is all zeros. Aborting (returning inf).')
            return float('inf')
       
        def nbow(document):
            d = zeros(vocab_len, dtype=double)
            nbow = dictionary.doc2bow(document)  # Word frequencies.
            doc_len = len(document)
            for idx, freq in nbow:
                d[idx] = freq / float(doc_len)  # Normalized word frequencies.
            return d

        # Compute nBOW representation of documents.
        d1 = nbow(document1)
        d2 = nbow(document2)

        # Compute WMD.
        return emd(d1, d2, distance_matrix)